In [1]:
import re
import numpy as np
from wrapper_my_radex import myradex_wrapper as wrapper 
phy_c_cgs=299792458e2
phy_c_SI=299792458
phy_k_cgs=1.3806503e-16
phy_h_cgs=6.62606896e-27
phy_cm2erg=phy_h_cgs*phy_c_cgs
phy_cm2K=phy_cm2erg/phy_k_cgs
class colli_partner:
    def __init__(self,part_name):
        self.name=part_name
    def set_n_trans(self,n_trans):
        self.n_transitions=n_trans
    def set_n_T(self,n_T):
        self.n_T=n_T
    def set_colli_temp(self,col_temp):
        self.colli_temperature=col_temp
    def set_Cul(self,Cul):
        self.Cul=Cul
class mol_data:
    def __init__(self,mol_dir,mol_file):
        self.mol_dir=mol_dir
        self.mol_file=mol_file
        f=open(self.mol_dir+self.mol_file)
        file_list=re.split('\n',f.read())
        f.close()
        if_read=True
        self.mol_name=re.split(' ',file_list[1])
        self.n_level=int(file_list[5])
        self.level_energy=np.zeros(self.n_level)
        self.level_weight=np.zeros(self.n_level)
        nrow=7
        for i in range(self.n_level):
            line=re.split(' |\t',file_list[nrow+i])
            line=[item for item in line if item!='']
            self.level_energy[i]=float(line[1])*phy_cm2K
            self.level_weight[i]=float(line[2])
        nrow+=self.n_level+1
        self.n_transitions=int(file_list[nrow])
        nrow+=2
        self.rad_data=np.zeros([self.n_transitions,9]) # iup, ilow, Aul, freq, lambda, Eup, Elow, Bul, Blu
        for i in range(self.n_transitions):
            line=re.split(' |\t',file_list[nrow+i])
            line=[item for item in line if item!='']
            iu=int(line[1])
            il=int(line[2])
            self.rad_data[i,0]=iu
            self.rad_data[i,1]=il
            self.rad_data[i,2]=float(line[3])
            self.rad_data[i,3]=phy_c_cgs*(self.level_energy[iu-1]-self.level_energy[il-1])/phy_cm2K
            self.rad_data[i,4]=phy_c_SI/self.rad_data[i,3]*1e6  # micron
            self.rad_data[i,5]=self.level_energy[iu-1]
            self.rad_data[i,6]=self.level_energy[il-1]
            self.rad_data[i,7]=self.rad_data[i,2]/((2*phy_h_cgs/phy_c_cgs**2)*self.rad_data[i,3]**3)
            self.rad_data[i,8]=self.rad_data[i,7]*self.level_weight[iu-1]/self.level_weight[il-1]
        nrow+=self.n_transitions+1
        self.n_partner=int(file_list[nrow])
        nrow+=2
        self.colli_data={}
        for i in range(self.n_partner):
            line=re.split(' |-|\t|\+|:',file_list[nrow])
            line=[item for item in line if item!='']
            part_name=line[2]
            if part_name=='electron':
                part_name='e'
            elif part_name=='with':
                part_name=line[3]
            partner=colli_partner(part_name)
            partner.set_n_trans(int(file_list[nrow+2]))
            partner.set_n_T(int(file_list[nrow+4]))
            line=re.split(' |\t',file_list[nrow+6])
            line=np.array([item for item in line if item!=''])
            partner.set_colli_temp(line.astype(float))
            Cul=np.zeros([partner.n_transitions,partner.n_T+2])
            for j in range(partner.n_transitions):
                line=re.split(' |\t',file_list[nrow+8+j])
                line=np.array([item for item in line if item!=''])
                Cul[j]=line[1:].astype(float)
            partner.set_Cul(Cul.T)
            self.colli_data[part_name]=partner
            nrow+=9+partner.n_transitions
        self.data_shape=[self.n_level,self.n_transitions,self.n_partner]
        self.partner_names=list(self.colli_data.keys())
        self.colli_shape=[[self.colli_data[key].n_transitions,self.colli_data[key].n_T] for key in self.colli_data.keys()]
        self.level_data=np.concatenate([[self.level_energy],[self.level_weight]],axis=0).T
        colli_T=np.zeros([self.n_partner,max([self.colli_data[key].n_T for key in self.partner_names])])
        colli_Cul=np.zeros([self.n_partner,max([self.colli_data[key].n_T for key in self.partner_names])+2,max([self.colli_data[key].n_transitions for key in self.partner_names])])
        for i in range(self.n_partner):
            partner_name=self.partner_names[i]
            colli_T[i,:self.colli_data[partner_name].n_T]=self.colli_data[partner_name].colli_temperature
            colli_Cul[i,:self.colli_data[partner_name].n_T+2,:self.colli_data[partner_name].n_transitions]=self.colli_data[partner_name].Cul
        self.colli_T=colli_T
        self.colli_Cul=colli_Cul
        space=' '
        self.part_name_str=space.join(self.partner_names)
co_mol=mol_data('/home/zj/Documents/radex_mol/','co.dat')
hcn_mol=mol_data('/home/zj/Documents/radex_mol/','hcn.dat')
co_mol.colli_Cul.shape

(2, 27, 820)

In [2]:
def run_myradex(Tkin,NXcol,nH2,Tbg,molecule,ini_occ=[]):
    if len(ini_occ)==0:
        ini_occ=molecule.level_weight*np.exp(-molecule.level_energy/Tkin)
        ini_occ=ini_occ/sum(ini_occ)
    params = {'tkin': Tkin,
              'ncol_x_cgs': NXcol,
              'h2_density_cgs': nH2,
              'tbg':Tbg,
              'mol_name':molecule.mol_name[0],
              'data_shape':molecule.data_shape,
              'n_transition':molecule.data_shape[1],
              'partner_names':molecule.part_name_str,
              'colli_shape':molecule.colli_shape,
              'level_data':molecule.level_data,
              'rad_data':molecule.rad_data,
              'colli_t':molecule.colli_T,
              'colli_data':molecule.colli_Cul,
              'ini_occ':ini_occ}      #initial occupation
    Tb=wrapper.run_one_params(**params)
    return Tb
import time
start=time.time()
for i in range(1):
    Tb,f_occ=run_myradex(35,1e18,1e4,2.73,co_mol)
print(time.time()-start,'s')
print(f_occ)

0.005406379699707031 s
[7.7641003e-02 1.9872974e-01 2.4162064e-01 2.1080433e-01 1.4424445e-01
 7.9826087e-02 3.5382450e-02 1.0456235e-02 1.2497709e-03 4.2498945e-05
 2.5891943e-06 1.9247861e-07 1.3209565e-08 7.8054269e-10 3.9409628e-11
 1.7112094e-12 6.3230427e-14 2.0150573e-15 5.3144141e-17 1.6604220e-18
 3.4202086e-20 6.0231017e-22 9.0308205e-24 1.1352252e-25 1.2225596e-27
 1.1022741e-29 8.7589799e-32 6.3857440e-34 3.6859966e-36 1.7534230e-38
 7.7389510e-41 2.8726619e-43 1.4012985e-45 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]


In [3]:
start=time.time()
for i in range(1):
    Tb,f_occ=run_myradex(35,1e18,1e4,2.73,co_mol,f_occ)
print(time.time()-start,'s')
print(f_occ)

0.0016589164733886719 s
[7.81394541e-02 1.99904814e-01 2.42801353e-01 2.11357653e-01
 1.43921211e-01 7.88147748e-02 3.41723710e-02 9.73354559e-03
 1.11408369e-03 3.81419341e-05 2.41592602e-06 1.79924427e-07
 1.23565371e-08 7.27779059e-10 3.68161196e-11 1.59238757e-12
 5.89377654e-14 1.87443203e-15 4.94277077e-17 1.54742425e-18
 3.18748976e-20 5.61013188e-22 8.40910027e-24 1.05674143e-25
 1.13765404e-27 1.02551376e-29 8.14688174e-32 5.93815527e-34
 3.42713642e-36 1.63011341e-38 7.19370580e-41 2.67648007e-43
 1.40129846e-45 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
